# FULLSTACK OPEN-SOURCE LAKEHOUSE PLATFORM

## E-Commerce Event History Analysis (Sử dụng bộ dữ liệu eCommerce Events History in Cosmetics Shop trên Kanggle)

### Architecture Stack
| Layer | Technology | Port |
|-------|------------|------|
| Storage | MinIO | 9000, 9001 |
| Table Format | Apache Iceberg | - |
| Catalog | Iceberg REST | 8181 |
| Compute | Apache Spark | 8080, 8888 |
| Transformation | dbt | - |
| Serving | ClickHouse | 8123, 9440 |
| Visualization | Apache Superset | 8088 |

---
## STEP 1: Setup Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

try:
    spark.stop()
    print("✅ Stop previous SparkSession.")
except NameError:
    pass

spark = SparkSession.builder \
    .appName("Lakehouse ETL eCommerces data") \
    .getOrCreate()
print("✅Started new sparksession.")

print(f"Spark Version: {spark.version}")
print(f"Catalog: {spark.conf.get('spark.sql.defaultCatalog')}")
print(f"Master: {spark.conf.get('spark.master')}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/04 16:35:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/04 16:35:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


✅Started new sparksession.
Spark Version: 3.5.1
Catalog: lakehouse
Master: spark://spark-master:7077


In [2]:
# Create Medallion namespaces
spark.sql("CREATE NAMESPACE IF NOT EXISTS lakehouse.bronze")
spark.sql("CREATE NAMESPACE IF NOT EXISTS lakehouse.silver")
spark.sql("CREATE NAMESPACE IF NOT EXISTS lakehouse.gold")

spark.sql("SHOW NAMESPACES IN lakehouse").show()

+---------+
|namespace|
+---------+
|   bronze|
|     gold|
|   silver|
+---------+



---
## STEP 2: BRONZE LAYER - Raw Data Ingestion

- Read CSV raw data
- Add metadata columns: `_ingestion_time`, `_source_file`
- Write as Iceberg table

In [3]:
import time

start = time.time()

df_19_oct = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/home/iceberg/notebooks/archive/2019-Oct.csv")
df_19_nov = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/home/iceberg/notebooks/archive/2019-Nov.csv")
df_19_dec = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/home/iceberg/notebooks/archive/2019-Dec.csv")
df_20_jan = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/home/iceberg/notebooks/archive/2020-Jan.csv")
df_20_feb = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/home/iceberg/notebooks/archive/2020-Feb.csv")

end = time.time()
print(f"⏱️ Thời gian chạy: {end - start:.2f} giây")

df_raw = df_19_oct.union(df_19_nov).union(df_19_dec).union(df_20_jan).union(df_20_feb)

print(f"Total records: {df_raw.count():,}")
df_raw.printSchema()
df_raw.show(5, truncate=False)

⏱️ Thời gian chạy: 27.29 giây


Total records: 20,692,840
root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)

+-------------------+----------+----------+-------------------+-------------+------+-----+---------+------------------------------------+
|event_time         |event_type|product_id|category_id        |category_code|brand |price|user_id  |user_session                        |
+-------------------+----------+----------+-------------------+-------------+------+-----+---------+------------------------------------+
|2019-10-01 00:00:00|cart      |5773203   |1487580005134238553|NULL         |runail|2.62 |463240011|26dd6e6e-4dac-4778-8d2c-92e149dab885|
|2019-10-01 00:00:03|cart      |5773353   

In [4]:
# Add metadata columns
df_19_oct = df_19_oct \
    .withColumn("_ingestion_time", current_timestamp()) \
    .withColumn("_source_file", lit("2019-Oct.csv"))
df_19_nov = df_19_nov \
    .withColumn("_ingestion_time", current_timestamp()) \
    .withColumn("_source_file", lit("2019-Nov.csv"))
df_19_dec = df_19_dec \
    .withColumn("_ingestion_time", current_timestamp()) \
    .withColumn("_source_file", lit("2019-Dec.csv"))
df_20_jan = df_20_jan \
    .withColumn("_ingestion_time", current_timestamp()) \
    .withColumn("_source_file", lit("2020-Jan.csv"))
df_20_feb = df_20_feb \
    .withColumn("_ingestion_time", current_timestamp()) \
    .withColumn("_source_file", lit("2020-Feb.csv"))

df_bronze = df_19_oct.union(df_19_nov).union(df_19_dec).union(df_20_jan).union(df_20_feb)

start = time.time()
# Write to Bronze layer
df_bronze.writeTo("lakehouse.bronze.ecommerce_events") \
    .tableProperty("write.format.default", "parquet") \
    .tableProperty("write.parquet.compression-codec", "snappy") \
    .createOrReplace()

end = time.time()
print(f"⏱️ Thời gian chạy: {end - start:.2f} giây")

print("BRONZE layer created!")
spark.sql("SELECT COUNT(*) as total FROM lakehouse.bronze.ecommerce_events").show()

⏱️ Thời gian chạy: 32.05 giây
BRONZE layer created!
+--------+
|   total|
+--------+
|20692840|
+--------+



---
## STEP 3: SILVER LAYER - Cleaned Data with Partitioning

- Parse timestamps
- Handle NULL values
- **Partition by event_date** (optimize query performance)

In [5]:
start = time.time()

# Transform Bronze -> Silver
df_silver = spark.table("lakehouse.bronze.ecommerce_events") \
    .withColumn("event_timestamp", to_timestamp(col("event_time"), "yyyy-MM-dd HH:mm:ss 'UTC'")) \
    .withColumn("event_date", to_date(col("event_timestamp"))) \
    .withColumn("event_hour", hour(col("event_timestamp"))) \
    .withColumn("brand", coalesce(col("brand"), lit("unknown"))) \
    .withColumn("category_code", coalesce(col("category_code"), lit("uncategorized"))) \
    .withColumn("price", col("price").cast("decimal(10,2)")) \
    .withColumn("_processed_at", current_timestamp()) \
    .drop("event_time", "_ingestion_time", "_source_file")

end = time.time()
print(f"⏱️ Thời gian chạy: {end - start:.2f} giây")
df_silver.printSchema()

⏱️ Thời gian chạy: 0.12 giây
root
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = false)
 |-- brand: string (nullable = false)
 |-- price: decimal(10,2) (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- event_timestamp: timestamp (nullable = true)
 |-- event_date: date (nullable = true)
 |-- event_hour: integer (nullable = true)
 |-- _processed_at: timestamp (nullable = false)



In [6]:
start = time.time()

# Write with PARTITIONING by date
df_silver.writeTo("lakehouse.silver.ecommerce_events_cleaned") \
    .tableProperty("write.format.default", "parquet") \
    .partitionedBy("event_date") \
    .createOrReplace()

end = time.time()
print(f"⏱️ Thời gian chạy: {end - start:.2f} giây")
print("SILVER layer created with partitioning!")

# Verify partitions
spark.sql("SELECT DISTINCT event_date FROM lakehouse.silver.ecommerce_events_cleaned ORDER BY event_date").show()

⏱️ Thời gian chạy: 17.95 giây
SILVER layer created with partitioning!


+----------+
|event_date|
+----------+
|2019-10-01|
|2019-10-02|
|2019-10-03|
|2019-10-04|
|2019-10-05|
|2019-10-06|
|2019-10-07|
|2019-10-08|
|2019-10-09|
|2019-10-10|
|2019-10-11|
|2019-10-12|
|2019-10-13|
|2019-10-14|
|2019-10-15|
|2019-10-16|
|2019-10-17|
|2019-10-18|
|2019-10-19|
|2019-10-20|
+----------+
only showing top 20 rows



---
## STEP 4: GOLD LAYER - Aggregated Tables

Create business-level aggregations with **partitioning** and **sorting**.

In [7]:
start = time.time()

# Gold 1: Daily Sales Summary (with partitioning)
df_daily = spark.sql("""
    SELECT 
        event_date,
        COUNT(*) AS total_events,
        COUNT(CASE WHEN event_type = 'view' THEN 1 END) AS views,
        COUNT(CASE WHEN event_type = 'cart' THEN 1 END) AS add_to_cart,
        COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) AS purchases,
        CAST(SUM(CASE WHEN event_type = 'purchase' THEN price ELSE 0 END) AS DECIMAL(12,2)) AS revenue,
        COUNT(DISTINCT user_id) AS unique_users,
        ROUND(COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) * 100.0 / 
              NULLIF(COUNT(CASE WHEN event_type = 'view' THEN 1 END), 0), 2) AS conversion_rate,
        current_timestamp() AS _aggregated_at
    FROM lakehouse.silver.ecommerce_events_cleaned
    GROUP BY event_date
    ORDER BY event_date
""")

df_daily.writeTo("lakehouse.gold.daily_sales_summary") \
    .tableProperty("write.format.default", "parquet") \
    .partitionedBy("event_date") \
    .createOrReplace()

end = time.time()
print(f"⏱️ Thời gian chạy: {end - start:.2f} giây")

print("Gold: daily_sales_summary created!")
spark.table("lakehouse.gold.daily_sales_summary").show()

⏱️ Thời gian chạy: 6.34 giây
Gold: daily_sales_summary created!
+----------+------------+-----+-----------+---------+--------+------------+---------------+--------------------+
|event_date|total_events|views|add_to_cart|purchases| revenue|unique_users|conversion_rate|      _aggregated_at|
+----------+------------+-----+-----------+---------+--------+------------+---------------+--------------------+
|2019-10-03|      124847|56934|      35745|     8865|43380.98|       16323|          15.57|2026-02-04 16:37:...|
|2019-10-04|      115612|53674|      31347|     7562|35887.15|       14732|          14.09|2026-02-04 16:37:...|
|2019-10-01|      142414|61209|      46916|     8476|43497.17|       19230|          13.85|2026-02-04 16:37:...|
|2019-10-02|      201068|76497|      89124|     9100|45746.20|       33859|          11.90|2026-02-04 16:37:...|
|2019-10-07|      181451|75899|      66954|     9376|46837.58|       26427|          12.35|2026-02-04 16:37:...|
|2019-10-08|      148944|64948| 

In [8]:
start = time.time()
# Gold 2: Brand Performance (sorted for efficient queries)
df_brand = spark.sql("""
    SELECT 
        brand,
        COUNT(*) AS total_events,
        COUNT(CASE WHEN event_type = 'view' THEN 1 END) AS views,
        COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) AS purchases,
        CAST(SUM(CASE WHEN event_type = 'purchase' THEN price ELSE 0 END) AS DECIMAL(12,2)) AS revenue,
        CAST(AVG(CASE WHEN event_type = 'purchase' THEN price END) AS DECIMAL(10,2)) AS avg_order_value,
        COUNT(DISTINCT user_id) AS unique_customers,
        current_timestamp() AS _aggregated_at
    FROM lakehouse.silver.ecommerce_events_cleaned
    WHERE brand != 'unknown'
    GROUP BY brand
    ORDER BY revenue DESC
""")

# Write with sort order for optimized queries
df_brand.writeTo("lakehouse.gold.brand_performance") \
    .tableProperty("write.format.default", "parquet") \
    .tableProperty("write.distribution-mode", "hash") \
    .createOrReplace()

end = time.time()
print(f"⏱️ Thời gian chạy: {end - start:.2f} giây")

print("Gold: brand_performance created!")
spark.table("lakehouse.gold.brand_performance").show(10)

⏱️ Thời gian chạy: 4.36 giây
Gold: brand_performance created!
+--------+------------+------+---------+---------+---------------+----------------+--------------------+
|   brand|total_events| views|purchases|  revenue|avg_order_value|unique_customers|      _aggregated_at|
+--------+------------+------+---------+---------+---------------+----------------+--------------------+
|  runail|     1528908|608857|   111408|343433.19|           3.08|          227164|2026-02-04 16:37:...|
| grattol|      852591|426257|    49542|266295.94|           5.38|          123453|2026-02-04 16:37:...|
|   irisk|     1033852|409737|    73806|223903.38|           3.03|          187621|2026-02-04 16:37:...|
|     uno|      250377|121177|    17586|190719.46|          10.84|           63592|2026-02-04 16:37:...|
|  strong|       60713| 50925|      850|151941.80|         178.76|           24736|2026-02-04 16:37:...|
|  masura|      861763|342247|    49751|139764.86|           2.81|           97435|2026-02-04 16:3

In [9]:
start = time.time()

# Gold 3: Hourly Traffic Pattern
df_hourly = spark.sql("""
    SELECT 
        event_hour,
        COUNT(*) AS total_events,
        COUNT(CASE WHEN event_type = 'view' THEN 1 END) AS views,
        COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) AS purchases,
        CAST(SUM(CASE WHEN event_type = 'purchase' THEN price ELSE 0 END) AS DECIMAL(12,2)) AS revenue,
        current_timestamp() AS _aggregated_at
    FROM lakehouse.silver.ecommerce_events_cleaned
    GROUP BY event_hour
    ORDER BY event_hour
""")

df_hourly.writeTo("lakehouse.gold.hourly_traffic") \
    .tableProperty("write.format.default", "parquet") \
    .createOrReplace()

end = time.time()
print(f"⏱️ Thời gian chạy: {end - start:.2f} giây")

print("Gold: hourly_traffic created!")
spark.table("lakehouse.gold.hourly_traffic").show(24)

⏱️ Thời gian chạy: 2.26 giây
Gold: hourly_traffic created!
+----------+------------+------+---------+---------+--------------------+
|event_hour|total_events| views|purchases|  revenue|      _aggregated_at|
+----------+------------+------+---------+---------+--------------------+
|         0|      202824| 89823|    11857| 55647.47|2026-02-04 16:37:...|
|         1|      184093| 76937|    11941| 53548.94|2026-02-04 16:37:...|
|         2|      202669| 84941|    12306| 58054.87|2026-02-04 16:37:...|
|         3|      283222|122096|    15500| 75796.41|2026-02-04 16:37:...|
|         4|      398557|181733|    22023|108355.57|2026-02-04 16:37:...|
|         5|      603889|273928|    35310|172329.76|2026-02-04 16:37:...|
|         6|      814449|377012|    50802|255409.26|2026-02-04 16:37:...|
|         7|      968572|455206|    62334|311343.85|2026-02-04 16:37:...|
|         8|     1066729|502937|    70323|362003.61|2026-02-04 16:37:...|
|         9|     1133492|529162|    76504|385431.64|2

In [10]:
start = time.time()

# Gold 4: Daily Sales by Category
df_category = spark.sql("""
    SELECT 
        event_date,
        category_code,
        COUNT(*) AS total_events,
        COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) AS purchases,
        CAST(SUM(CASE WHEN event_type = 'purchase' THEN price ELSE 0 END) AS DECIMAL(12,2)) AS revenue,
        current_timestamp() AS _aggregated_at
    FROM lakehouse.silver.ecommerce_events_cleaned
    WHERE category_code != 'uncategorized'
    GROUP BY event_date, category_code
    ORDER BY event_date, revenue DESC
""")

df_category.writeTo("lakehouse.gold.daily_sales_by_category") \
    .tableProperty("write.format.default", "parquet") \
    .partitionedBy("event_date") \
    .createOrReplace()

end = time.time()
print(f"⏱️ Thời gian chạy: {end - start:.2f} giây")

print("Gold: daily_sales_by_category created!")
spark.table("lakehouse.gold.daily_sales_by_category").show(10)

⏱️ Thời gian chạy: 3.18 giây
Gold: daily_sales_by_category created!
+----------+--------------------+------------+---------+-------+--------------------+
|event_date|       category_code|total_events|purchases|revenue|      _aggregated_at|
+----------+--------------------+------------+---------+-------+--------------------+
|2019-10-03|appliances.enviro...|         913|       32|1204.67|2026-02-04 16:37:...|
|2019-10-03|       apparel.glove|         196|       24| 179.69|2026-02-04 16:37:...|
|2019-10-03|furniture.bathroo...|         125|        7| 172.64|2026-02-04 16:37:...|
|2019-10-03|     accessories.bag|         153|        3| 127.38|2026-02-04 16:37:...|
|2019-10-03| stationery.cartrige|         497|       42|  94.52|2026-02-04 16:37:...|
|2019-10-03|furniture.living_...|         234|        1|  82.54|2026-02-04 16:37:...|
|2019-10-03|accessories.cosme...|          63|        5|  27.80|2026-02-04 16:37:...|
|2019-10-03|furniture.living_...|           3|        0|   0.00|2026-02-

---
## STEP 4.1: Verify All Tables & Locations

In [11]:
print("=" * 60)
print("BRONZE TABLES")
print("=" * 60)
spark.sql("SHOW TABLES IN lakehouse.bronze").show()

print("=" * 60)
print("SILVER TABLES")
print("=" * 60)
spark.sql("SHOW TABLES IN lakehouse.silver").show()

print("=" * 60)
print("GOLD TABLES")
print("=" * 60)
spark.sql("SHOW TABLES IN lakehouse.gold").show()

BRONZE TABLES
+---------+----------------+-----------+
|namespace|       tableName|isTemporary|
+---------+----------------+-----------+
|   bronze|ecommerce_events|      false|
+---------+----------------+-----------+

SILVER TABLES
+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|   silver|ecommerce_events_...|      false|
+---------+--------------------+-----------+

GOLD TABLES
+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|     gold|   brand_performance|      false|
|     gold|daily_sales_by_ca...|      false|
|     gold| daily_sales_summary|      false|
|     gold|      hourly_traffic|      false|
+---------+--------------------+-----------+



In [12]:
# Check file locations in MinIO
tables = [
    "lakehouse.bronze.ecommerce_events",
    "lakehouse.silver.ecommerce_events_cleaned",
    "lakehouse.gold.daily_sales_summary",
    "lakehouse.gold.brand_performance",
    "lakehouse.gold.hourly_traffic",
    "lakehouse.gold.daily_sales_by_category"
]

print("Table Locations in MinIO:")
print("=" * 80)
for table in tables:
    try:
        location = spark.sql(f"DESCRIBE EXTENDED {table}") \
            .filter(col("col_name") == "Location") \
            .select("data_type").collect()[0][0]
        print(f"{table}")
        print(f"  -> {location}")
    except:
        print(f"{table} - Not found")

Table Locations in MinIO:
lakehouse.bronze.ecommerce_events
  -> s3://lakehouse/bronze/ecommerce_events
lakehouse.silver.ecommerce_events_cleaned
  -> s3://lakehouse/silver/ecommerce_events_cleaned
lakehouse.gold.daily_sales_summary
  -> s3://lakehouse/gold/daily_sales_summary
lakehouse.gold.brand_performance
  -> s3://lakehouse/gold/brand_performance
lakehouse.gold.hourly_traffic
  -> s3://lakehouse/gold/hourly_traffic
lakehouse.gold.daily_sales_by_category
  -> s3://lakehouse/gold/daily_sales_by_category


---
## STEP 5: CLICKHOUSE Integration (Zero-Copy)

### Run these commands to exec ClickHouse:
```bash
docker exec -it lakehouse-clickhouse clickhouse-client --user admin --password password
```
After that, run code  below:

In [13]:
# ================================================================
# CLICKHOUSE SQL COMMANDS
# ================================================================

clickhouse_sql_commands = [
    # 1. Create database
    "CREATE DATABASE IF NOT EXISTS lakehouse",
    
    # 2. Daily Sales Summary (Zero-Copy from MinIO)
    """CREATE OR REPLACE TABLE lakehouse.daily_sales_summary
    ENGINE = S3('http://minio:9000/lakehouse/gold/daily_sales_summary/data/*/*.parquet',
                 'admin', 'password', 'Parquet')""",
    
    # 3. Brand Performance
    """CREATE OR REPLACE TABLE lakehouse.brand_performance
    ENGINE = S3('http://minio:9000/lakehouse/gold/brand_performance/data/*.parquet',
                 'admin', 'password', 'Parquet')""",
    
    # 4. Hourly Traffic
    """CREATE OR REPLACE TABLE lakehouse.hourly_traffic
    ENGINE = S3('http://minio:9000/lakehouse/gold/hourly_traffic/data/*.parquet',
                 'admin', 'password', 'Parquet')""",
    
    # 5. Daily Sales by Category
    """CREATE OR REPLACE TABLE lakehouse.daily_sales_by_category
    ENGINE = S3('http://minio:9000/lakehouse/gold/daily_sales_by_category/data/*/*.parquet',
                 'admin', 'password', 'Parquet')"""
]

print("✅ Đã chuẩn bị", len(clickhouse_sql_commands), "câu lệnh SQL cho ClickHouse")


✅ Đã chuẩn bị 5 câu lệnh SQL cho ClickHouse


In [14]:
import requests
import time

def run_clickhouse_query(query, host='clickhouse', port=8123, user='admin', password='password'):
    """Chạy SQL query thông qua HTTP API của ClickHouse"""
    url = f'http://{host}:{port}/'
    try:
        response = requests.post(
            url,
            auth=(user, password),
            data=query.encode('utf-8'),
            params={'database': 'default'}
        )
        response.raise_for_status()
        return response.text
    except Exception as e:
        raise Exception(f"Lỗi: {str(e)}")

try:
    print("🔗 Đang kết nối đến ClickHouse qua HTTP API...")
    
    # Test connection
    result = run_clickhouse_query("SELECT version()")
    print(f"✅ Kết nối thành công! ClickHouse version: {result.strip()}")
    
    # Chạy từng câu lệnh
    for i, sql in enumerate(clickhouse_sql_commands, 1):
        try:
            print(f"\n[{i}/{len(clickhouse_sql_commands)}] Đang chạy: {sql[:50]}...")
            run_clickhouse_query(sql)
            print(f"✅ Hoàn thành!")
            time.sleep(0.5)  # Delay nhỏ giữa các lệnh
        except Exception as e:
            print(f"❌ Lỗi: {str(e)}")
    
    # Verify tables
    tables = [
        'lakehouse.daily_sales_summary',
        'lakehouse.brand_performance',
        'lakehouse.hourly_traffic',
        'lakehouse.daily_sales_by_category'
    ]
    
    for table in tables:
        result = run_clickhouse_query(f"SELECT COUNT(*) FROM {table}")
        print(f"{table.split('.')[-1]}: {result.strip()} rows")
    
except Exception as e:
    print(f" Lỗi: {str(e)}")


🔗 Đang kết nối đến ClickHouse qua HTTP API...
✅ Kết nối thành công! ClickHouse version: 24.1.8.22

[1/5] Đang chạy: CREATE DATABASE IF NOT EXISTS lakehouse...
✅ Hoàn thành!

[2/5] Đang chạy: CREATE OR REPLACE TABLE lakehouse.daily_sales_summ...
✅ Hoàn thành!

[3/5] Đang chạy: CREATE OR REPLACE TABLE lakehouse.brand_performanc...
✅ Hoàn thành!

[4/5] Đang chạy: CREATE OR REPLACE TABLE lakehouse.hourly_traffic
 ...
✅ Hoàn thành!

[5/5] Đang chạy: CREATE OR REPLACE TABLE lakehouse.daily_sales_by_c...
✅ Hoàn thành!
daily_sales_summary: 304 rows
brand_performance: 546 rows
hourly_traffic: 48 rows
daily_sales_by_category: 2950 rows


---
## STEP 6: SUPERSET Dashboard Setup

### Access
- URL: http://localhost:8088
- Username: `admin`
- Password: `admin`

### Add ClickHouse Connection
1. Settings -> Database Connections -> + Database
2. Select: **ClickHouse Connect**
3. Connection :
```
-Host: clickhouse

-Port: 8123

-Database name: lakehouse

-Username: admin

-Password: password

-Display Name: ClickHouse Lakehouse

-SSL: (optional)

click Connect/Test Connection.
```

### Create 3 Required Charts

| Chart | Type | Dataset | Metrics |
|-------|------|---------|--------|
| Daily Revenue Trend | Line Chart | daily_sales_summary | revenue by event_date |
| Top Brands | Bar Chart | brand_performance | revenue by brand |
| Hourly Traffic | Area Chart | hourly_traffic | views, purchases by event_hour |

### Create Dashboard
1. Dashboards -> + Dashboard
2. Add the 3 charts
3. Arrange and save

---
## STEP 7: dbt Transformation (Alternative)

### Run dbt commands:
```bash
# Enter dbt container
docker exec -it lakehouse-dbt bash

# Run transformations
cd /usr/app/dbt/lakehouse_dbt
dbt debug
dbt run
```

dbt models are located in:
- `/dbt/lakehouse_dbt/models/silver/` - Silver layer transformations
- `/dbt/lakehouse_dbt/models/gold/` - Gold layer aggregations